<img src='https://user-images.githubusercontent.com/6457691/90080969-0f758d00-dd47-11ea-8191-fa12fd2054a7.png' width = '200' align = 'right'>

## AIB / SECTION 3 / SC33x 

---



# Major Neural Network Architectures

이번 한 주간 CNN, U-Net, Autoencoder, GAN 등 다양한 주요 신경망 구조들에 대해서 배워봤습니다. 오늘은 그 모델들을 복습하는 시간을 가지도록 하겠습니다. 이 SC는 **신경망의 다양한 구조에 대한 이해와 지식**을 평가합니다. **모델을 높은 정확도를 가지도록 학습 시킬 수 있는지를 평가하려는 것이 아닙니다.**

아래의 방식들은 복잡한 연산을 요구합니다. 모든 파트의 문제들은 어떤 환경에서라도 (e.g. 로컬 주피터, Google Colab, etc.) 5-10분 내외로 결과값이 나오도록 제작이 됐기 때문에 만일 결과값을 도출하는데 그 이상의 시간이 걸린다면 여러분의 접근 방식을 재점검해보시기 바랍니다.

---

## 1. CNN

### 이미지 분류
Keras와  [ResNet50v2](https://www.tensorflow.org/api_docs/python/tf/keras/applications/resnet_v2) (pre-trained)을 활용하여 `im_frog` 폴더에 있는 이미지 중 어떤 이미지에 개구리가 있는지 찾는 이미지 분류 모델을 작동시켜 보겠습니다.

<img align="left" src="https://d3i6fh83elv35t.cloudfront.net/newshour/app/uploads/2017/03/GettyImages-654745934-1024x687.jpg" width=400>

## 2.1 Resnet$V_2$을 사용하기 위해서 전처리 함수를 사용하여 이미지를 전처리 하고 이미지들의 사이즈를 재조정하는 함수를 정의해봅시다.

In [162]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


- **파일 경로와 이름을 불러옵니다.**

In [163]:
import os

frog_dir = "/content/drive/MyDrive/SC33x/im_frog"

In [164]:
"""Hint : os 패키지 내에 파일 이름을 불러올 수 있는 메서드를 사용합니다."""

filenames = os.listdir(frog_dir)

In [165]:
filenames

['cristiane-teston-bcnfJvEYm1Y-unsplash.jpg',
 'zdenek-machacek-HYTwWSE5ztw-unsplash (1).jpg',
 'priscilla-du-preez-oWJcgqjFb6I-unsplash.jpg',
 'serenity-mitchell-tUDSHkd6rYQ-unsplash.jpg',
 'yanna-zissiadou-SV-aMgliWNs-unsplash.jpg',
 'elizabeth-explores-JZybccsrB-0-unsplash.jpg',
 'joel-henry-Rcvf6-n1gc8-unsplash.jpg',
 'mche-lee-j-P8z4EOgyQ-unsplash.jpg',
 'saturday_sun-_q37Ca0Ll4o-unsplash.jpg',
 'jared-evans-VgRnolD7OIw-unsplash.jpg',
 'ed-van-duijn-S1zA6AR50X8-unsplash.jpg',
 'matthew-kosloski-sYkr-M78H6w-unsplash.jpg',
 'marcus-neto-fH_DOdTt-pA-unsplash.jpg',
 'drew-brown-VBvoy5gofWg-unsplash.jpg',
 'jacky-watt-92W5jPbOj48-unsplash.jpg']

- **필요한 라이브러리 import 후 이미지를 불러와 예측하는 함수를 정의합니다.**

In [166]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

In [167]:
def load_and_preprocessing(base_dir, name, model):
    """
    이미지 1장을 받아 모델로 예측한 뒤
    가장 확률이 높은 클래스 번호를 출력하는 함수입니다.
    
    Hint:
        1. 경로에서 이미지를 불러옵니다.
        2. array의 값을 직접 나누어 픽셀 값을 정규화합니다.
        
    Args:
        base_dir : 이미지 파일이 있는 경로입니다.
        name : 이미지 파일의 이름입니다.
        model : 예측에 사용할 모델입니다.
    """
    image_path = os.path.join(base_dir, name)
    image = load_img(image_path, target_size=(224, 224))
    input_arr = img_to_array(image) / 255.0
    
    input_arr = np.array([input_arr])
    predictions = model.predict(input_arr)
    predict_class = np.argmax(predictions, axis=1) 
    print(predict_class[0])

    return predict_class[0]

## 2.2 ResNet50v2 모델을 사용해 이미지 분류(예측)를 진행합니다. 예측 결과는 자유롭게 출력해봅니다.
> 참고: `ResNet50v2`는 "frog"로 예측하지 않습니다. "frog"의 label은 "bullfrog, treefrog, tailed frog"입니다

In [168]:
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, decode_predictions, preprocess_input

resnetV2 = ResNet50V2(weights='imagenet', include_top = True, input_shape=(224,224,3))
resnetV2.summary()

Model: "resnet50v2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_36 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_36[0][0]']               
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                         

- **기존에 정의한 함수(`load_and_preprocessing`)를 사용하여 이미지 분류를 진행하여봅니다.**

In [169]:
# 출력되는 클래스의 번호를 모두 리스트에 저장하여 predict_class 에 할당합니다.

predict_class = [load_and_preprocessing(frog_dir, filename, resnetV2) for filename in filenames]

1/1 [==============================] - 1s 1s/step
309
1/1 [==============================] - 0s 22ms/step
31
1/1 [==============================] - 0s 21ms/step
58
1/1 [==============================] - 0s 23ms/step
985
1/1 [==============================] - 0s 22ms/step
113
1/1 [==============================] - 0s 24ms/step
807
1/1 [==============================] - 0s 24ms/step
31
1/1 [==============================] - 0s 25ms/step
985
1/1 [==============================] - 0s 24ms/step
58
1/1 [==============================] - 0s 27ms/step
32
1/1 [==============================] - 0s 25ms/step
308
1/1 [==============================] - 0s 31ms/step
397
1/1 [==============================] - 0s 22ms/step
738
1/1 [==============================] - 0s 29ms/step
30
1/1 [==============================] - 0s 24ms/step
868


- **클래스의 이름을 다운받아 출력하여봅니다.**

In [170]:
!git clone https://github.com/anishathalye/imagenet-simple-labels.git

fatal: destination path 'imagenet-simple-labels' already exists and is not an empty directory.


In [171]:
import json

with open ('/content/imagenet-simple-labels/imagenet-simple-labels.json') as f:
    labels = json.load(f)

for i in predict_class:
    print(labels[i])

bee
tree frog
water snake
daisy
snail
solar thermal collector
tree frog
daisy
water snake
tailed frog
fly
pufferfish
pot
American bullfrog
tray


---

## 2. U-Net

Lecture Note에서는 U-Net의 백본(backbone)모델로 `MobileNetV2`를 사용하여 segmentation을 수행하였습니다.<br/>
이번 SC에서는 ResNet50을 백본으로 하여 같은 문제를 풀어보세요.

참고로 resnet에서의 block은 아래 예시의 3개의 레이어를 참조하여 만들어주세요.<br/>
예시는 16x16 까지만 나타나 있지만 Lecture Note 와 같이 4x4까지 만들어 주어야 모델을 완성할 수 있습니다.

**(최소 `model.complie`까지 진행해야 점수가 부여됩니다!!)**

```
    'conv1_relu', # 64x64
    'conv2_block3_out', # 32x32 
    'conv3_block4_out', # 16x16
```

In [172]:
import tensorflow as tf

base_model = tf.keras.applications.ResNet50(input_shape=[128, 128, 3], include_top=False)

layer_names = [
    'conv1_conv',
    'conv2_block3_out',
    'conv3_block4_out',
    'conv4_block6_out',
    'conv5_block3_out',
]

layers = [base_model.get_layer(name).output for name in layer_names]
down_stack = tf.keras.Model(inputs=base_model.input, outputs=layers)

down_stack.trainable = False

94765736/94765736 [==============================] - 5s 0us/step


In [173]:
base_model.compile(optimizer="adam", loss="sparse_categorical_crossentropy",metrics=['accuracy'])

## 3. Free Response
> 동료들에게 설명한다고 생각하고 간단하게 몇 문장으로 작성하세요.

### 3.1 딥러닝이 왜 중요하다고 생각하시나요?

실생활을 더욱 편리하게 만들기 때문에 입니다

### 3.2 딥러닝의 다양한 분야 중에서 좀 더 심도있게 배우고 싶은 분야는 무엇인가요? 왜 그렇게 생각하시나요?

Chat GPT에 많은 도움을 받고 있는 만큼 자연어 처리에 관심이 있지만 더 배우고 싶은 분야는 이미지입니다

### 3.3 인공지능이 우리 사회에서 대체할 수 있는 직업이 무엇이라고 생각하시나요?


대체할 수 없는 직업을 찾는게 더 빠를 것 같습니다.

### 3.4 반대로 인공지능 때문에, 딥러닝 때문에 더 생겨날 직업은 무엇이 있을까요?


인간이 할 수 없는 일,비도덕,비위생적인 일을 대신 해주는 직업이 나오지 않을까요?

### 3.5 여러분이 생각하실 때 Strong AI라고 불리우는 [일반 인공지능 (Artificial General Intelligence)](https://ko.wikipedia.org/wiki/%EC%9D%B8%EA%B3%B5_%EC%9D%BC%EB%B0%98_%EC%A7%80%EB%8A%A5)을 개발해내는 것이 가능할 것 같나요? 왜 그렇게 생각하시나요?

가능 할 것 같습니다 인공지능은 인간을 모방하는 행위 이기 때문에 언젠가는 가능 할 것이라고 생각합니다.

### 3.6 Coutinous learning에 대해서 조사해보고 인지한 부분에 대해서 작성해보세요.

Continuous learning은 모델이 처음 학습한 데이터와는 다른 데이터를 지속적으로 추가하고, 이를 통해 모델을 개선하는 것입니다.

# Advanced Goals: 3점을 받기 위해서는 아래의 조건 중에서 3개 이상을 달성하셔야 합니다

### 1
    - 개구리 이외의 다른 객체를 탐지하는 모델을 만들어보세요 (예: 물고기)
    - 이미지를 예측한 label과 같이 출력해보세요
    - 예측 모델을 함수로 만들어 보세요 (물론 주석도 잘 되어있어야 합니다)
### 2
    - U-Net 을 직접 구현하여 동일한 문제를 수행해보세요. 
### 3
    - 작성한 답안을 몇 문장보다 조금 더 상세하게 작성해보세요
    - 왜 그렇게 생각하게 되었는 지 관련 근거를 서술하세요.
### 추가과제
    - GAN을 이용한 프로젝트를 새롭게 구현해보세요.